# Matrix Factorization

We will implement matrix factorization here, both manually and maybe through TensorFlow.

## Theory Behind Matrix Factorization: Problem Statement

We will give a brief rundown, nothing too complicated.

The main **mathematical** statement that we will tackle with matrix factorization is as follows:

We have some matrix $A$ (this could be a tabular dataset or anything else). Now the *caveat* is that $A$ has **missing entries** that do not know about. We want to find two matrices $U$ and $V$, so that their product:

$$U \cdot V^{T} \approx A$$

So really, in terms of an optimization problem, we want to minimize each entry of the matrix $E$:

$$E = A - U\cdot V^{T}$$

It really helps to write this in index notation to represent each component so that we are not dealing with matrices directly. Let $E_{ij} := e_{ij}$, then, writing out the above expression in matrix form, we see that:

$$e_{ij} = a_{ij} - \sum_{k}u_{ik}v_{jk}^{T} = a_{ij} - \sum_{k}u_{ik}v_{kj}$$

We are now only working with real numbers! We can now adopt the traditional MSE loss for each entry.

$$e_{ij}^{2} = \left(a_{ij} - \sum_{k}u_{ik}v_{kj}\right)^{2}$$

or to make it look simpler, we can write $\hat{a}_{ij} = \sum_{k}u_{ik}v_{kj}$, and we now have:

$$e_{ij}^{2} = (a_{ij} - \hat{a}_{ij})^{2}$$

We are now minimizing this expression for all $(i,j) \in rows(A), cols(A))$. **For all intents and purposes**, this is all we really need to know to continue.

## Theory of Matrix Factorization: Machine Learning

However, we can continue our derivation of our equations, as this will help us in our implementation. We take the gradient (derivative) of our loss above, with respect to the nonzero components of both $U$ and $V$ (**Notice that we are not training weights and biases, but just the entries of the matrices $U$ and $V$!!!**):

$$\frac{\partial e_{ij}^{2}}{\partial u_{ik}} = \frac{\partial e_{ij}^{2}}{\partial e_{ij}}\frac{\partial e_{ij}}{\partial u_{ik}} = -2e_{ij}v_{kj} $$

Likewise, for the other component:

$$\frac{\partial e_{ij}^{2}}{\partial v_{ik}} = -2e_{ij}u_{ik}$$

We now do the **optimization step** (i.e. the *Gradient Descent Step*):

$$u_{ik} \longmapsto u_{ik} - \lambda \frac{\partial e_{ij}^{2}}{\partial u_{ik}} = u_{ik} + 2\lambda e_{ij}v_{kj}$$

$$v_{kj} \longmapsto v_{kj} - \lambda \frac{\partial e_{ij}^{2}}{\partial v_{kj}} = v_{kj} + 2\lambda e_{ij}u_{ik}$$

## Business Use of Matrix Factorization

The business problem we are trying to solve is the following:

We are given a table of **customers** and **products** they interacted with, however, some customers do not leave ratings on the product that they interacted with. In this case, how do we find these missing ratings?

Furthermore, let us say that we track certain features for the customers and products. For example:

Let $U$ denote the customers, and let $V$ denote items bought at a food mart. Let us say that $U$, $V$ contains **2 features**, sweet and savory, and let $A$ be the rating of sweet or savory items (from 1-5) by a customer that purchased them. Sweet and savory are what we refer to as **latent features**, and they relate customers to products as they are keys that merge customers and products. 

Now assume $A$ is the following customer-product interaction table...

$$A = \begin{pmatrix} 
4 & 0 & 3 & 2 & 2 & 1\\[3pt]
0 & 5 & 3 & 4 & 5 & 1\\[3pt]
5 & 5 & 0 & 0 & 3 & 5\\[3pt]
3 & 3 & 4 & 3 & 2 & 0\\[3pt]
2 & 4 & 2 & 4 & 5 & 1\\[3pt]
0 & 0 & 5 & 3 & 3 & 4
\end{pmatrix}$$

Then we want to find $U$ and $V$ such that:

- $U$ encodes the latent features (sweet or savory) of the customers, and $V$ does the same for the products.
- $U \cdot V^{T} \approx A$.

Thus, we are looking for $U$, $V$ such that:

$$A \approx U\cdot V^{T}$$

Now refer to the previous section for the mathematical details of our machine learning problem.


## Manual Implementation of Matrix Factorization

By "manual implementation", we mean *implementation in python with only numpy*. This is possible because we are using the following loss:

$$\mathrm{Loss} = \mathrm{MSE} + \mathrm{Regularization}$$

This loss goes component-by-component:

$$e_{ij}^{2} = (a_{ij} - \hat{a}_{ij})^{2}$$

The total loss is

$$\hat{L} = \sum_{i,j} e_{ij}^{2}$$

We also add a regularization term:

$$L = \hat{L} + \sum_{k=1}^{N}\beta\left(u_{ik}^{2} + v_{jk}^{2}\right)$$

For $N$ latent features. This is to prevent overfitting.

So we have a closed-form expression for the gradient of the loss in this case. If we wanted to implement more complicated loss functions, we will likely need to use an automatic differentiation package.

In [1]:
import numpy as np

In [2]:
#ARGUMENTS
# A - ground truth matrix
# U - product 1
# V - product 2
# number of features
# epochs - number of epochs to train
# lambda - learning rate
# beta - regularization parameter

def matrix_factorization(A, U, V, k, steps, lmbda=0.0002, beta = 0.02):
    V = V.T

    for epoch in range(steps):
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                # we are only looking to minimize the loss on the already-filled entries
                if A[i][j] > 0:
                    # error
                    eij = A[i][j] - np.dot(U[i, :], V[:, j])
                    
                    for K in range(k):
                        # gradient descent step
                        U[i][K] = U[i][K] + lmbda * (2*eij * V[K][j] - beta*U[i][K])
                        V[K][j] = V[K][j] + lmbda * (2*eij * U[i][K] - beta*V[K][j])
        
        # e is our loss (i.e. our e_{ij}^{2})
        e = 0
        
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                if A[i][j] > 0:
                    # update the loss
                    e = e + (A[i][j] - np.dot(U[i,:], V[:,j]))**2
                    
                    for K in range(k):
                        # put regularization term on loss so it doesn't overfit on training data
                        e = e + (beta/2) * (U[i][K]**2 + V[K][j]**2)
                     
        # give print output of training (print only every 10 epochs)
        if epoch % 10 == 0:
            print(f'Epoch {epoch + 1} : Total Loss {e:.3f}')
        if e < 0.001:
            break
    return U, V.T

In [3]:
A = np.array([
    [5,3,2,1,0],
    [3,0,4,2,1],
    [4,0,0,1,0],
    [0,0,2,4,0],
    [5,3,0,0,1]
])

m = A.shape[0]
n = A.shape[1]
user_features = 2

np.random.seed(24)

# generate matrices with entries of random numbers 0 - 1
U = np.random.rand(m, user_features)
V = np.random.rand(n, user_features)

# check entries of matrices

print('U is:\n', U)
print('V is:\n', V)

# perform factorization

U_hat, V_hat = matrix_factorization(A, U, V, user_features, 200)

print('U_hat is:\n', U_hat)
print('V_hat is:\n', V_hat)

## check if the product results in something close to A

print('A_hat is:\n', np.dot(U_hat, V_hat.T))

U is:
 [[0.9600173  0.69951205]
 [0.99986729 0.2200673 ]
 [0.36105635 0.73984099]
 [0.99645573 0.31634698]
 [0.13654458 0.38398001]]
V is:
 [[0.32051928 0.36641475]
 [0.70965156 0.90014243]
 [0.53411544 0.24729376]
 [0.67180656 0.56172911]
 [0.54255988 0.8934476 ]]
Epoch 1 : Total Loss 99.275
Epoch 11 : Total Loss 96.767
Epoch 21 : Total Loss 94.202
Epoch 31 : Total Loss 91.584
Epoch 41 : Total Loss 88.918
Epoch 51 : Total Loss 86.207
Epoch 61 : Total Loss 83.459
Epoch 71 : Total Loss 80.680
Epoch 81 : Total Loss 77.878
Epoch 91 : Total Loss 75.059
Epoch 101 : Total Loss 72.233
Epoch 111 : Total Loss 69.408
Epoch 121 : Total Loss 66.595
Epoch 131 : Total Loss 63.801
Epoch 141 : Total Loss 61.036
Epoch 151 : Total Loss 58.311
Epoch 161 : Total Loss 55.633
Epoch 171 : Total Loss 53.012
Epoch 181 : Total Loss 50.455
Epoch 191 : Total Loss 47.970
U_hat is:
 [[1.2427563  0.99404657]
 [1.32958304 0.4449768 ]
 [0.53087359 0.91554011]
 [1.23935835 0.47920855]
 [0.51268872 0.8134178 ]]
V_hat is

This does not look good after 200 epochs! Let's try more epochs, and if that doesn't work, let's increase the learning rate.

In [4]:
U_hat, V_hat = matrix_factorization(A, U, V, user_features, 5000)

print('A_hat is:\n', np.dot(U_hat, V_hat.T))

Epoch 1 : Total Loss 45.564
Epoch 11 : Total Loss 43.244
Epoch 21 : Total Loss 41.013
Epoch 31 : Total Loss 38.877
Epoch 41 : Total Loss 36.838
Epoch 51 : Total Loss 34.898
Epoch 61 : Total Loss 33.060
Epoch 71 : Total Loss 31.323
Epoch 81 : Total Loss 29.687
Epoch 91 : Total Loss 28.150
Epoch 101 : Total Loss 26.711
Epoch 111 : Total Loss 25.366
Epoch 121 : Total Loss 24.114
Epoch 131 : Total Loss 22.950
Epoch 141 : Total Loss 21.870
Epoch 151 : Total Loss 20.871
Epoch 161 : Total Loss 19.947
Epoch 171 : Total Loss 19.095
Epoch 181 : Total Loss 18.310
Epoch 191 : Total Loss 17.587
Epoch 201 : Total Loss 16.923
Epoch 211 : Total Loss 16.312
Epoch 221 : Total Loss 15.751
Epoch 231 : Total Loss 15.236
Epoch 241 : Total Loss 14.763
Epoch 251 : Total Loss 14.328
Epoch 261 : Total Loss 13.929
Epoch 271 : Total Loss 13.563
Epoch 281 : Total Loss 13.225
Epoch 291 : Total Loss 12.914
Epoch 301 : Total Loss 12.628
Epoch 311 : Total Loss 12.364
Epoch 321 : Total Loss 12.120
Epoch 331 : Total Los

Epoch 2901 : Total Loss 5.527
Epoch 2911 : Total Loss 5.526
Epoch 2921 : Total Loss 5.526
Epoch 2931 : Total Loss 5.525
Epoch 2941 : Total Loss 5.524
Epoch 2951 : Total Loss 5.523
Epoch 2961 : Total Loss 5.523
Epoch 2971 : Total Loss 5.522
Epoch 2981 : Total Loss 5.521
Epoch 2991 : Total Loss 5.521
Epoch 3001 : Total Loss 5.520
Epoch 3011 : Total Loss 5.519
Epoch 3021 : Total Loss 5.519
Epoch 3031 : Total Loss 5.518
Epoch 3041 : Total Loss 5.517
Epoch 3051 : Total Loss 5.517
Epoch 3061 : Total Loss 5.516
Epoch 3071 : Total Loss 5.516
Epoch 3081 : Total Loss 5.515
Epoch 3091 : Total Loss 5.515
Epoch 3101 : Total Loss 5.514
Epoch 3111 : Total Loss 5.513
Epoch 3121 : Total Loss 5.513
Epoch 3131 : Total Loss 5.512
Epoch 3141 : Total Loss 5.512
Epoch 3151 : Total Loss 5.511
Epoch 3161 : Total Loss 5.511
Epoch 3171 : Total Loss 5.510
Epoch 3181 : Total Loss 5.510
Epoch 3191 : Total Loss 5.509
Epoch 3201 : Total Loss 5.509
Epoch 3211 : Total Loss 5.508
Epoch 3221 : Total Loss 5.508
Epoch 3231

After training for long enough, we see that the existing interactions are predicted quite well. To see the average error of each entry in $A_{hat}$ corresponding to NONZEROS in $A$, we can just use the following:

In [5]:
A_hat = np.dot(U_hat, V_hat.T)
A_hat

array([[4.98447014, 2.97009182, 1.54598166, 1.50219606, 0.9902487 ],
       [2.99605858, 2.12761136, 3.17161116, 2.87072417, 0.95654322],
       [3.98158649, 2.34293891, 1.04127825, 1.03001406, 0.75980498],
       [4.52329661, 2.9474781 , 3.05471784, 2.81272953, 1.16479036],
       [4.96573884, 3.01061644, 1.87870719, 1.79363369, 1.0410779 ]])

In [6]:
A

array([[5, 3, 2, 1, 0],
       [3, 0, 4, 2, 1],
       [4, 0, 0, 1, 0],
       [0, 0, 2, 4, 0],
       [5, 3, 0, 0, 1]])

### Write a Method to Evaluate our Output

Since this is a regression task, we will be evaluating the performance of our model through (root) mean squared error.

In [7]:
# have function to evaluate total loss in our prediction

def evaluation_metrics(A, A_hat, return_nonzero_indices = False, return_rmse = True):
    # find all nonzero indices of ground truth matrix
    nonzero_ind = [(i,j) for i in range(A.shape[0]) for j in range(A.shape[1]) if A[i,j] > 0]
    
    # compute the losses in each ground truth matrix
    avg_error = sum([(A_hat[i][j] - A[i][j])**2 for i, j in nonzero_ind])/len(nonzero_ind)
    rmse = np.sqrt(avg_error)
    
    # set a flag for if we want to return the nonzero indices or not
    li_results = []
    if return_rmse:
        li_results.append(avg_error)
        li_results.append(rmse)
    else:
        li_results.append(avg_error)
    
    if return_nonzero_indices:
        li_results.append(nonzero_indices)
    else:
        pass
    
    return li_results

In [8]:
mse, rmse = evaluation_metrics(A, A_hat)

print(f'The mean squared error of all predictions is {mse:.3f}.')
print(f'The root mean squared error of all predictions is {rmse:.3f}.')

The mean squared error of all predictions is 0.295.
The root mean squared error of all predictions is 0.544.


Therefore, our matrix factorization algorithm is very good at predicting the existing entries. Let us see what the predictions were for the empty entries.

### Write Method to Predict the Zero Entries

In [9]:
# have a function predict the missing entries of our ground truth matrix

def mat_predict(A, A_hat, conv_to_int = True, zip_indices = False):
    # find zero indices of A (our missing entries that we are predicting)
    zero_ind = [(i,j) for i in range(A.shape[0]) for j in range(A.shape[1]) if A[i][j] == 0]
    
    # find predictions
    og_pred = [A_hat[i,j] for i,j in zero_ind]
    
    # WARNING: FOR VERY LARGE AND SPARSE MATRICES, I DO NOT RECOMMEND DOING THIS. JUST LOOK AT THE OUTPUT
    if zip_indices:
        for prediction, indices in zip(pred, zero_ind):
            print(f'The interaction at row, column {indices} is {prediction}.')
            
    # convert output to integers
    if conv_to_int:
        # if the rounded A_hat has entries greater than the max of A, then set it to floor, otherwise, round it to integer
        int_prediction = np.where(A_hat > np.max(A), np.floor(A_hat), np.round(A_hat, decimals=0))
        
        # return integer predictions at indices where 0's originally were in the ground truth A
        return [int_prediction[i,j] for i,j in zero_ind]
    else:
        # return the original floating point predictions from the algorithm output
        
        return og_pred

In [10]:
mat_predict(A, A_hat)

[1.0, 2.0, 2.0, 1.0, 1.0, 5.0, 3.0, 1.0, 2.0, 2.0]

In [11]:
# find zero indices of A
zero_ind = [(i,j) for i in range(A.shape[0]) for j in range(A.shape[1]) if A[i][j] == 0]

zero_ind

[(0, 4),
 (1, 1),
 (2, 1),
 (2, 2),
 (2, 4),
 (3, 0),
 (3, 1),
 (3, 4),
 (4, 2),
 (4, 3)]

In [12]:
predictions = [[A_hat[i][j], f'At index:{(i,j)}'] for i,j in zero_ind]

predictions

[[0.990248697667072, 'At index:(0, 4)'],
 [2.1276113626791973, 'At index:(1, 1)'],
 [2.3429389126641804, 'At index:(2, 1)'],
 [1.0412782499755882, 'At index:(2, 2)'],
 [0.7598049813295886, 'At index:(2, 4)'],
 [4.523296605425644, 'At index:(3, 0)'],
 [2.947478096798725, 'At index:(3, 1)'],
 [1.1647903631780367, 'At index:(3, 4)'],
 [1.8787071873760717, 'At index:(4, 2)'],
 [1.793633687301854, 'At index:(4, 3)']]

To take it one step further, we could even round these up or down in order to give a good prediction for what these interactions *would have* been rated.

In [13]:
rounded_predictions = [[int(np.round(A_hat[i][j], decimals=0)), f'At index:{(i,j)}'] for i,j in zero_ind]

rounded_predictions

[[1, 'At index:(0, 4)'],
 [2, 'At index:(1, 1)'],
 [2, 'At index:(2, 1)'],
 [1, 'At index:(2, 2)'],
 [1, 'At index:(2, 4)'],
 [5, 'At index:(3, 0)'],
 [3, 'At index:(3, 1)'],
 [1, 'At index:(3, 4)'],
 [2, 'At index:(4, 2)'],
 [2, 'At index:(4, 3)']]

**Non-Square Matrix Test Case**

In [14]:
A = np.array([
    [5,3,0,1],
    [3,2,0,0],
    [0,1,4,2],
    [1,0,0,5]
])

m, n = A.shape[0], A.shape[1]

user_features = 2

np.random.seed(42)

U_1 = np.random.rand(m, user_features)
V_1 = np.random.rand(n, user_features)

# check entries of matrices

print('U is:\n', U_1)
print('V is:\n', V_1)

# perform factorization

U_hat, V_hat = matrix_factorization(A, U_1, V_1, user_features, 5000)

A_hat = np.dot(U_hat, V_hat.T)

print('U_hat is:\n', U_hat)
print('V_hat is:\n', V_hat)

## check if the product results in something close to A
print('A is:\n', A)
print('A_hat is:\n', A_hat)


U is:
 [[0.37454012 0.95071431]
 [0.73199394 0.59865848]
 [0.15601864 0.15599452]
 [0.05808361 0.86617615]]
V is:
 [[0.60111501 0.70807258]
 [0.02058449 0.96990985]
 [0.83244264 0.21233911]
 [0.18182497 0.18340451]]
Epoch 1 : Total Loss 70.905
Epoch 11 : Total Loss 69.765
Epoch 21 : Total Loss 68.610
Epoch 31 : Total Loss 67.441
Epoch 41 : Total Loss 66.260
Epoch 51 : Total Loss 65.068
Epoch 61 : Total Loss 63.867
Epoch 71 : Total Loss 62.659
Epoch 81 : Total Loss 61.447
Epoch 91 : Total Loss 60.231
Epoch 101 : Total Loss 59.014
Epoch 111 : Total Loss 57.799
Epoch 121 : Total Loss 56.587
Epoch 131 : Total Loss 55.380
Epoch 141 : Total Loss 54.182
Epoch 151 : Total Loss 52.994
Epoch 161 : Total Loss 51.819
Epoch 171 : Total Loss 50.657
Epoch 181 : Total Loss 49.513
Epoch 191 : Total Loss 48.386
Epoch 201 : Total Loss 47.280
Epoch 211 : Total Loss 46.195
Epoch 221 : Total Loss 45.134
Epoch 231 : Total Loss 44.098
Epoch 241 : Total Loss 43.087
Epoch 251 : Total Loss 42.103
Epoch 261 : Tot

Epoch 2651 : Total Loss 3.099
Epoch 2661 : Total Loss 3.091
Epoch 2671 : Total Loss 3.084
Epoch 2681 : Total Loss 3.078
Epoch 2691 : Total Loss 3.071
Epoch 2701 : Total Loss 3.065
Epoch 2711 : Total Loss 3.059
Epoch 2721 : Total Loss 3.053
Epoch 2731 : Total Loss 3.047
Epoch 2741 : Total Loss 3.041
Epoch 2751 : Total Loss 3.036
Epoch 2761 : Total Loss 3.031
Epoch 2771 : Total Loss 3.026
Epoch 2781 : Total Loss 3.021
Epoch 2791 : Total Loss 3.016
Epoch 2801 : Total Loss 3.011
Epoch 2811 : Total Loss 3.007
Epoch 2821 : Total Loss 3.002
Epoch 2831 : Total Loss 2.998
Epoch 2841 : Total Loss 2.994
Epoch 2851 : Total Loss 2.990
Epoch 2861 : Total Loss 2.986
Epoch 2871 : Total Loss 2.982
Epoch 2881 : Total Loss 2.979
Epoch 2891 : Total Loss 2.975
Epoch 2901 : Total Loss 2.972
Epoch 2911 : Total Loss 2.968
Epoch 2921 : Total Loss 2.965
Epoch 2931 : Total Loss 2.961
Epoch 2941 : Total Loss 2.958
Epoch 2951 : Total Loss 2.955
Epoch 2961 : Total Loss 2.952
Epoch 2971 : Total Loss 2.949
Epoch 2981

In [15]:
mse, rmse = evaluation_metrics(A, A_hat)

print(f'The mean squared error of all predictions is {mse:.3f}.')
print(f'The root mean squared error of all predictions is {rmse:.3f}.')

mat_predict(A, A_hat)

The mean squared error of all predictions is 0.176.
The root mean squared error of all predictions is 0.420.


[5.0, 3.0, 2.0, 4.0, 3.0, 1.0]

**Try a Large Matrix**

In [16]:
# np.random.seed(1)

# A = np.random.randint(0, 6, size=(100,100))

# m, n = A.shape[0], A.shape[1]

# user_features = 2

# U_2 = np.random.rand(m, user_features)
# V_2 = np.random.rand(n, user_features)

# # check entries of matrices

# print('U is:\n', U_2)
# print('V is:\n', V_2)

# # perform factorization

# U_hat, V_hat = matrix_factorization(A, U_2, V_2, user_features, 5000)

# A_hat = np.dot(U_hat, V_hat.T)

# print('U_hat is:\n', U_hat)
# print('V_hat is:\n', V_hat)

# ## check if the product results in something close to A
# print('A is:\n', A)
# print('A_hat is:\n', A_hat)


**Note: This takes a really long time to train and evaluate. It took over 15 minutes to train and evaluate a 100 by 100 matrix with 2 latent features. Most datasets that we will put in in a business situation will be WELL over that amount so this current implementation is not feasible for large datasets.**

## Matrix Factorization Implementation for Sparse Matrices

We will investigate how to perform matrix factorization for sparse matrices, as most matrices we will work with for the purpose of matrix factorization are large and sparse (if it were large but not sparse, we would have most of our information anyways, so this technique wouldn't be super useful).

### Sparse Matrices

The definition of a sparse matrix is very imprecise, but for our purposes, we define a sparse matrix as any matrix that stores a 0 (or any other value that represents the absence of meaningful data) in at most $\max(m,n)$ entries in an $m\times n$ matrix.

We can use scipy's csr_matrix class to get the nonzero elements of a sparse matrix. Let us use a toy example!

In [17]:
from scipy.sparse import csr_matrix

In [18]:
x = np.array([
    [0,1,0,0,0,0,0],
    [0,0,5,0,0,1,0],
    [0,3,0,0,0,0,1],
    [4,0,0,2,0,0,0]    
          ])

x

array([[0, 1, 0, 0, 0, 0, 0],
       [0, 0, 5, 0, 0, 1, 0],
       [0, 3, 0, 0, 0, 0, 1],
       [4, 0, 0, 2, 0, 0, 0]])

In [19]:
# print items in csr_matrix

for item in csr_matrix(x):
    print(item)

  (0, 1)	1
  (0, 2)	5
  (0, 5)	1
  (0, 1)	3
  (0, 6)	1
  (0, 0)	4
  (0, 3)	2


So a csr_matrix object only stores nonzero elements (the meaningful data entries) in memory!

This is important for our matrix factorization algorithm implementation here, because recall that **our loss computation only involves NONZERO elements** anyways! So we do not need any information about the matrix with the 0 entries!

We can pick out the nonzero indices of the matrix with **csr_matrix's nonzero method**.

In [20]:
csr_matrix(x).nonzero()

(array([0, 1, 1, 2, 2, 3, 3], dtype=int32),
 array([1, 2, 5, 1, 6, 0, 3], dtype=int32))

In [21]:
# index the nonzero elements through extracting the indices
li_ind = []

for x_ind, y_ind in zip(csr_matrix(x).nonzero()[0], csr_matrix(x).nonzero()[1]):
    li_ind.append((x_ind, y_ind))

li_ind

[(0, 1), (1, 2), (1, 5), (2, 1), (2, 6), (3, 0), (3, 3)]

In [22]:
# access all elements of our original array
for i, j in zip(csr_matrix(x).nonzero()[0], csr_matrix(x).nonzero()[1]):
    print(x[i,j])

1
5
1
3
1
4
2


### Matrix Factorization for Sparse Matrices

In [23]:
# This function is different from the previous one because we do not need to traverse each array separately
# we isolate and extract all of the nonzero entries from the matrix in linear time (at worst O(n) for ground truth mxn)
# and then we only loop through epochs and the number of latent features

# ARGUMENTS
# A - ground truth matrix (sparse numpy array)
# U - product 1
# V - product 2
# number of features
# epochs - number of epochs to train
# lambda - learning rate
# beta - regularization parameter

def matrix_factorization_sparse(A, U, V, k, steps, lmbda=0.0002, beta = 0.02):
    # transpose V to make matrix multiplication work
    V = V.T
    
    # convert A, a sparse numpy array, into a sparse scipy matrix object
    sparse_A = csr_matrix(A)
    
    for epoch in range(steps):
        # initialize total loss
        e = 0
        for i, j in zip(sparse_A.nonzero()[0], sparse_A.nonzero()[1]):
            # component-wise error
            eij = A[i,j] - np.dot(U[i,:], V[:,j])
            # total error
            e += (eij**2)
            
            for K in range(k):
                # gradient descent step for each entry, with regularization term
                U[i,K] = U[i,K] + lmbda * (2 * eij * V[K,j] - beta*U[i,K])
                V[K,j] = V[K,j] + lmbda * (2 * eij * U[i,K] - beta*V[K,j])
                
                # sum up total loss with regularization terms
                e += 0.5 * beta * (U[i][K]**2 + V[K][j]**2)

        # give print output of training (print only every 10 epochs)
        if epoch % 10 == 0:
            print(f'Epoch {epoch + 1} : Total Loss {e:.3f}')
        if e < 0.001:
            break
    return U, V.T

### Test on a Small Example

In [24]:
A = np.array([
    [5,3,2,1,0],
    [3,0,4,2,1],
    [4,0,0,1,0],
    [0,0,2,4,0],
    [5,3,0,0,1]
])

m = A.shape[0]
n = A.shape[1]
user_features = 2

np.random.seed(24)

# generate matrices with entries of random numbers 0 - 1
U = np.random.rand(m, user_features)
V = np.random.rand(n, user_features)

# check entries of matrices

print('U is:\n', U)
print('V is:\n', V)

# perform factorization

U_hat, V_hat = matrix_factorization_sparse(A, U, V, user_features, 5000)

print('U_hat is:\n', U_hat)
print('V_hat is:\n', V_hat)

## check if the product results in something close to A

print('A_hat is:\n', np.dot(U_hat, V_hat.T))

U is:
 [[0.9600173  0.69951205]
 [0.99986729 0.2200673 ]
 [0.36105635 0.73984099]
 [0.99645573 0.31634698]
 [0.13654458 0.38398001]]
V is:
 [[0.32051928 0.36641475]
 [0.70965156 0.90014243]
 [0.53411544 0.24729376]
 [0.67180656 0.56172911]
 [0.54255988 0.8934476 ]]
Epoch 1 : Total Loss 99.448
Epoch 11 : Total Loss 96.944
Epoch 21 : Total Loss 94.384
Epoch 31 : Total Loss 91.770
Epoch 41 : Total Loss 89.107
Epoch 51 : Total Loss 86.400
Epoch 61 : Total Loss 83.655
Epoch 71 : Total Loss 80.879
Epoch 81 : Total Loss 78.078
Epoch 91 : Total Loss 75.261
Epoch 101 : Total Loss 72.436
Epoch 111 : Total Loss 69.612
Epoch 121 : Total Loss 66.798
Epoch 131 : Total Loss 64.003
Epoch 141 : Total Loss 61.238
Epoch 151 : Total Loss 58.510
Epoch 161 : Total Loss 55.830
Epoch 171 : Total Loss 53.205
Epoch 181 : Total Loss 50.644
Epoch 191 : Total Loss 48.155
Epoch 201 : Total Loss 45.745
Epoch 211 : Total Loss 43.419
Epoch 221 : Total Loss 41.183
Epoch 231 : Total Loss 39.040
Epoch 241 : Total Loss 36

Epoch 2691 : Total Loss 5.590
Epoch 2701 : Total Loss 5.588
Epoch 2711 : Total Loss 5.586
Epoch 2721 : Total Loss 5.585
Epoch 2731 : Total Loss 5.583
Epoch 2741 : Total Loss 5.581
Epoch 2751 : Total Loss 5.579
Epoch 2761 : Total Loss 5.578
Epoch 2771 : Total Loss 5.576
Epoch 2781 : Total Loss 5.575
Epoch 2791 : Total Loss 5.573
Epoch 2801 : Total Loss 5.571
Epoch 2811 : Total Loss 5.570
Epoch 2821 : Total Loss 5.569
Epoch 2831 : Total Loss 5.567
Epoch 2841 : Total Loss 5.566
Epoch 2851 : Total Loss 5.564
Epoch 2861 : Total Loss 5.563
Epoch 2871 : Total Loss 5.562
Epoch 2881 : Total Loss 5.561
Epoch 2891 : Total Loss 5.559
Epoch 2901 : Total Loss 5.558
Epoch 2911 : Total Loss 5.557
Epoch 2921 : Total Loss 5.556
Epoch 2931 : Total Loss 5.555
Epoch 2941 : Total Loss 5.553
Epoch 2951 : Total Loss 5.552
Epoch 2961 : Total Loss 5.551
Epoch 2971 : Total Loss 5.550
Epoch 2981 : Total Loss 5.549
Epoch 2991 : Total Loss 5.548
Epoch 3001 : Total Loss 5.547
Epoch 3011 : Total Loss 5.546
Epoch 3021

### Test Runtime

We will test the runtime of the code we implemented for the sparse data.

We will test both methods for the same amount of epochs and latent features, learning rate, etc. on the same data.

In [25]:
import timeit as t

**Regular Matrix Factorization**

In [26]:
start1 = t.default_timer()

U_hat, V_hat = matrix_factorization(A, U, V, user_features, 5000)

end1 = t.default_timer()

print('The runtime of the regular matrix factorization is ', end1 - start1, 'seconds.')

Epoch 1 : Total Loss 5.473
Epoch 11 : Total Loss 5.473
Epoch 21 : Total Loss 5.473
Epoch 31 : Total Loss 5.473
Epoch 41 : Total Loss 5.473
Epoch 51 : Total Loss 5.473
Epoch 61 : Total Loss 5.473
Epoch 71 : Total Loss 5.473
Epoch 81 : Total Loss 5.472
Epoch 91 : Total Loss 5.472
Epoch 101 : Total Loss 5.472
Epoch 111 : Total Loss 5.472
Epoch 121 : Total Loss 5.472
Epoch 131 : Total Loss 5.472
Epoch 141 : Total Loss 5.472
Epoch 151 : Total Loss 5.471
Epoch 161 : Total Loss 5.471
Epoch 171 : Total Loss 5.471
Epoch 181 : Total Loss 5.471
Epoch 191 : Total Loss 5.471
Epoch 201 : Total Loss 5.471
Epoch 211 : Total Loss 5.471
Epoch 221 : Total Loss 5.470
Epoch 231 : Total Loss 5.470
Epoch 241 : Total Loss 5.470
Epoch 251 : Total Loss 5.470
Epoch 261 : Total Loss 5.470
Epoch 271 : Total Loss 5.470
Epoch 281 : Total Loss 5.470
Epoch 291 : Total Loss 5.469
Epoch 301 : Total Loss 5.469
Epoch 311 : Total Loss 5.469
Epoch 321 : Total Loss 5.469
Epoch 331 : Total Loss 5.469
Epoch 341 : Total Loss 5.

Epoch 2991 : Total Loss 5.358
Epoch 3001 : Total Loss 5.357
Epoch 3011 : Total Loss 5.356
Epoch 3021 : Total Loss 5.355
Epoch 3031 : Total Loss 5.354
Epoch 3041 : Total Loss 5.353
Epoch 3051 : Total Loss 5.352
Epoch 3061 : Total Loss 5.351
Epoch 3071 : Total Loss 5.350
Epoch 3081 : Total Loss 5.349
Epoch 3091 : Total Loss 5.347
Epoch 3101 : Total Loss 5.346
Epoch 3111 : Total Loss 5.345
Epoch 3121 : Total Loss 5.344
Epoch 3131 : Total Loss 5.343
Epoch 3141 : Total Loss 5.342
Epoch 3151 : Total Loss 5.341
Epoch 3161 : Total Loss 5.340
Epoch 3171 : Total Loss 5.338
Epoch 3181 : Total Loss 5.337
Epoch 3191 : Total Loss 5.336
Epoch 3201 : Total Loss 5.335
Epoch 3211 : Total Loss 5.334
Epoch 3221 : Total Loss 5.333
Epoch 3231 : Total Loss 5.331
Epoch 3241 : Total Loss 5.330
Epoch 3251 : Total Loss 5.329
Epoch 3261 : Total Loss 5.328
Epoch 3271 : Total Loss 5.326
Epoch 3281 : Total Loss 5.325
Epoch 3291 : Total Loss 5.324
Epoch 3301 : Total Loss 5.323
Epoch 3311 : Total Loss 5.321
Epoch 3321

**Sparse Matrix Factorization**

In [27]:
start2 = t.default_timer()

U_hat_sparse, V_hat_sparse = matrix_factorization_sparse(A, U, V, user_features, 5000)

end2 = t.default_timer()

print('The runtime of the sparse matrix factorization is ', end2 - start2, 'seconds.')

Epoch 1 : Total Loss 4.899
Epoch 11 : Total Loss 4.895
Epoch 21 : Total Loss 4.891
Epoch 31 : Total Loss 4.886
Epoch 41 : Total Loss 4.882
Epoch 51 : Total Loss 4.878
Epoch 61 : Total Loss 4.873
Epoch 71 : Total Loss 4.869
Epoch 81 : Total Loss 4.864
Epoch 91 : Total Loss 4.860
Epoch 101 : Total Loss 4.856
Epoch 111 : Total Loss 4.851
Epoch 121 : Total Loss 4.847
Epoch 131 : Total Loss 4.842
Epoch 141 : Total Loss 4.838
Epoch 151 : Total Loss 4.833
Epoch 161 : Total Loss 4.828
Epoch 171 : Total Loss 4.824
Epoch 181 : Total Loss 4.819
Epoch 191 : Total Loss 4.815
Epoch 201 : Total Loss 4.810
Epoch 211 : Total Loss 4.805
Epoch 221 : Total Loss 4.800
Epoch 231 : Total Loss 4.796
Epoch 241 : Total Loss 4.791
Epoch 251 : Total Loss 4.786
Epoch 261 : Total Loss 4.781
Epoch 271 : Total Loss 4.776
Epoch 281 : Total Loss 4.772
Epoch 291 : Total Loss 4.767
Epoch 301 : Total Loss 4.762
Epoch 311 : Total Loss 4.757
Epoch 321 : Total Loss 4.752
Epoch 331 : Total Loss 4.747
Epoch 341 : Total Loss 4.

Epoch 2941 : Total Loss 3.103
Epoch 2951 : Total Loss 3.098
Epoch 2961 : Total Loss 3.092
Epoch 2971 : Total Loss 3.087
Epoch 2981 : Total Loss 3.082
Epoch 2991 : Total Loss 3.077
Epoch 3001 : Total Loss 3.072
Epoch 3011 : Total Loss 3.067
Epoch 3021 : Total Loss 3.062
Epoch 3031 : Total Loss 3.057
Epoch 3041 : Total Loss 3.052
Epoch 3051 : Total Loss 3.047
Epoch 3061 : Total Loss 3.042
Epoch 3071 : Total Loss 3.037
Epoch 3081 : Total Loss 3.032
Epoch 3091 : Total Loss 3.027
Epoch 3101 : Total Loss 3.022
Epoch 3111 : Total Loss 3.017
Epoch 3121 : Total Loss 3.012
Epoch 3131 : Total Loss 3.007
Epoch 3141 : Total Loss 3.002
Epoch 3151 : Total Loss 2.998
Epoch 3161 : Total Loss 2.993
Epoch 3171 : Total Loss 2.988
Epoch 3181 : Total Loss 2.983
Epoch 3191 : Total Loss 2.979
Epoch 3201 : Total Loss 2.974
Epoch 3211 : Total Loss 2.969
Epoch 3221 : Total Loss 2.965
Epoch 3231 : Total Loss 2.960
Epoch 3241 : Total Loss 2.956
Epoch 3251 : Total Loss 2.951
Epoch 3261 : Total Loss 2.946
Epoch 3271

There is not a significant difference on these, however, the $A$ that we used in the **above example is not sparse at all.** 14/25 entries are nonzero, so the sparsity is only 44%. **We will try this method on a significantly more sparse matrix.**

## Test on a Large Sparse Matrix

**This will be a 100 x 100 sparse matrix with 2 latent features, and it will be 99% sparse.**

In [28]:
# generate random sparse matrix
from scipy.sparse import random
from scipy.stats import randint
arr_rvs = randint(0, 6).rvs

x = random(100,100, density = 0.01, data_rvs = arr_rvs)

arr_100_100 = np.array(x.toarray())

arr_100_100

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
li_ind = [(i,j) for i in range(arr_100_100.shape[0]) for j in range(arr_100_100.shape[1]) if arr_100_100[i,j]!=0]

li_arr_100_100 = [arr_100_100[i,j] for i,j in li_ind]

li_arr_100_100, len(li_arr_100_100)

([4.0,
  5.0,
  4.0,
  2.0,
  1.0,
  2.0,
  1.0,
  5.0,
  1.0,
  1.0,
  1.0,
  1.0,
  3.0,
  4.0,
  3.0,
  5.0,
  5.0,
  2.0,
  5.0,
  4.0,
  1.0,
  3.0,
  3.0,
  5.0,
  2.0,
  1.0,
  4.0,
  4.0,
  2.0,
  2.0,
  1.0,
  1.0,
  3.0,
  4.0,
  3.0,
  3.0,
  2.0,
  3.0,
  2.0,
  3.0,
  1.0,
  3.0,
  1.0,
  2.0,
  5.0,
  5.0,
  5.0,
  3.0,
  4.0,
  4.0,
  3.0,
  1.0,
  4.0,
  5.0,
  3.0,
  5.0,
  5.0,
  1.0,
  5.0,
  1.0,
  1.0,
  4.0,
  1.0,
  4.0,
  2.0,
  2.0,
  2.0,
  3.0,
  3.0,
  1.0,
  4.0,
  2.0,
  2.0,
  1.0,
  3.0,
  1.0,
  3.0,
  4.0,
  4.0,
  2.0,
  1.0,
  1.0,
  3.0,
  3.0],
 84)

### Run the Sparse Matrix Factorization on our Sparse Matrix

In [30]:
np.random.seed(1)

m, n = arr_100_100.shape[0], arr_100_100.shape[1]

user_features = 2

U_3 = np.random.rand(m, user_features)
V_3 = np.random.rand(n, user_features)

# check entries of matrices

print('U is:\n', U_3)
print('V is:\n', V_3)

# perform factorization
start3 = t.default_timer()

U_hat, V_hat = matrix_factorization_sparse(arr_100_100, U_3, V_3, user_features, 5000)

end3 = t.default_timer()

A_hat = np.dot(U_hat, V_hat.T)

print('U_hat is:\n', U_hat)
print('V_hat is:\n', V_hat)

## check if the product results in something close to A
print('A is:\n', arr_100_100)
print('A_hat is:\n', A_hat)

U is:
 [[4.17022005e-01 7.20324493e-01]
 [1.14374817e-04 3.02332573e-01]
 [1.46755891e-01 9.23385948e-02]
 [1.86260211e-01 3.45560727e-01]
 [3.96767474e-01 5.38816734e-01]
 [4.19194514e-01 6.85219500e-01]
 [2.04452250e-01 8.78117436e-01]
 [2.73875932e-02 6.70467510e-01]
 [4.17304802e-01 5.58689828e-01]
 [1.40386939e-01 1.98101489e-01]
 [8.00744569e-01 9.68261576e-01]
 [3.13424178e-01 6.92322616e-01]
 [8.76389152e-01 8.94606664e-01]
 [8.50442114e-02 3.90547832e-02]
 [1.69830420e-01 8.78142503e-01]
 [9.83468338e-02 4.21107625e-01]
 [9.57889530e-01 5.33165285e-01]
 [6.91877114e-01 3.15515631e-01]
 [6.86500928e-01 8.34625672e-01]
 [1.82882773e-02 7.50144315e-01]
 [9.88861089e-01 7.48165654e-01]
 [2.80443992e-01 7.89279328e-01]
 [1.03226007e-01 4.47893526e-01]
 [9.08595503e-01 2.93614148e-01]
 [2.87775339e-01 1.30028572e-01]
 [1.93669579e-02 6.78835533e-01]
 [2.11628116e-01 2.65546659e-01]
 [4.91573159e-01 5.33625451e-02]
 [5.74117605e-01 1.46728575e-01]
 [5.89305537e-01 6.99758360e-01]
 [1

Epoch 591 : Total Loss 145.687
Epoch 601 : Total Loss 141.717
Epoch 611 : Total Loss 137.869
Epoch 621 : Total Loss 134.143
Epoch 631 : Total Loss 130.534
Epoch 641 : Total Loss 127.039
Epoch 651 : Total Loss 123.657
Epoch 661 : Total Loss 120.384
Epoch 671 : Total Loss 117.218
Epoch 681 : Total Loss 114.156
Epoch 691 : Total Loss 111.194
Epoch 701 : Total Loss 108.331
Epoch 711 : Total Loss 105.563
Epoch 721 : Total Loss 102.887
Epoch 731 : Total Loss 100.301
Epoch 741 : Total Loss 97.802
Epoch 751 : Total Loss 95.387
Epoch 761 : Total Loss 93.053
Epoch 771 : Total Loss 90.798
Epoch 781 : Total Loss 88.618
Epoch 791 : Total Loss 86.512
Epoch 801 : Total Loss 84.476
Epoch 811 : Total Loss 82.508
Epoch 821 : Total Loss 80.606
Epoch 831 : Total Loss 78.766
Epoch 841 : Total Loss 76.987
Epoch 851 : Total Loss 75.265
Epoch 861 : Total Loss 73.600
Epoch 871 : Total Loss 71.988
Epoch 881 : Total Loss 70.427
Epoch 891 : Total Loss 68.916
Epoch 901 : Total Loss 67.451
Epoch 911 : Total Loss 66

Epoch 3331 : Total Loss 7.897
Epoch 3341 : Total Loss 7.877
Epoch 3351 : Total Loss 7.858
Epoch 3361 : Total Loss 7.838
Epoch 3371 : Total Loss 7.819
Epoch 3381 : Total Loss 7.800
Epoch 3391 : Total Loss 7.781
Epoch 3401 : Total Loss 7.763
Epoch 3411 : Total Loss 7.744
Epoch 3421 : Total Loss 7.726
Epoch 3431 : Total Loss 7.708
Epoch 3441 : Total Loss 7.690
Epoch 3451 : Total Loss 7.672
Epoch 3461 : Total Loss 7.654
Epoch 3471 : Total Loss 7.636
Epoch 3481 : Total Loss 7.619
Epoch 3491 : Total Loss 7.602
Epoch 3501 : Total Loss 7.585
Epoch 3511 : Total Loss 7.568
Epoch 3521 : Total Loss 7.551
Epoch 3531 : Total Loss 7.534
Epoch 3541 : Total Loss 7.518
Epoch 3551 : Total Loss 7.501
Epoch 3561 : Total Loss 7.485
Epoch 3571 : Total Loss 7.469
Epoch 3581 : Total Loss 7.453
Epoch 3591 : Total Loss 7.437
Epoch 3601 : Total Loss 7.422
Epoch 3611 : Total Loss 7.406
Epoch 3621 : Total Loss 7.391
Epoch 3631 : Total Loss 7.376
Epoch 3641 : Total Loss 7.361
Epoch 3651 : Total Loss 7.346
Epoch 3661

In [31]:
print(f'The runtime of the sparse matrix factorization on a 100 x 100, 99% sparse matrix is {end3 - start3} seconds.')

mse, rmse = evaluation_metrics(arr_100_100, A_hat)

print(f'The mean squared error of all predictions is {mse:.3f}.')
print(f'The root mean squared error of all predictions is {rmse:.3f}.')

The runtime of the sparse matrix factorization on a 100 x 100, 99% sparse matrix is 6.4532393999397755 seconds.
The mean squared error of all predictions is 0.008.
The root mean squared error of all predictions is 0.089.


### Run Regular Matrix Factorization on Sparse Matrix

Just for comparison, let us check how much more efficient our sparse matrix factorization is compared to the regular method.

In [32]:
np.random.seed(1)

m, n = arr_100_100.shape[0], arr_100_100.shape[1]

user_features = 2

U_3 = np.random.rand(m, user_features)
V_3 = np.random.rand(n, user_features)

# check entries of matrices

print('U is:\n', U_3)
print('V is:\n', V_3)

# perform factorization
start4 = t.default_timer()

U_hat, V_hat = matrix_factorization(arr_100_100, U_3, V_3, user_features, 5000)

end4 = t.default_timer()

A_hat = np.dot(U_hat, V_hat.T)

print('U_hat is:\n', U_hat)
print('V_hat is:\n', V_hat)

## check if the product results in something close to A
print('A is:\n', arr_100_100)
print('A_hat is:\n', A_hat)

U is:
 [[4.17022005e-01 7.20324493e-01]
 [1.14374817e-04 3.02332573e-01]
 [1.46755891e-01 9.23385948e-02]
 [1.86260211e-01 3.45560727e-01]
 [3.96767474e-01 5.38816734e-01]
 [4.19194514e-01 6.85219500e-01]
 [2.04452250e-01 8.78117436e-01]
 [2.73875932e-02 6.70467510e-01]
 [4.17304802e-01 5.58689828e-01]
 [1.40386939e-01 1.98101489e-01]
 [8.00744569e-01 9.68261576e-01]
 [3.13424178e-01 6.92322616e-01]
 [8.76389152e-01 8.94606664e-01]
 [8.50442114e-02 3.90547832e-02]
 [1.69830420e-01 8.78142503e-01]
 [9.83468338e-02 4.21107625e-01]
 [9.57889530e-01 5.33165285e-01]
 [6.91877114e-01 3.15515631e-01]
 [6.86500928e-01 8.34625672e-01]
 [1.82882773e-02 7.50144315e-01]
 [9.88861089e-01 7.48165654e-01]
 [2.80443992e-01 7.89279328e-01]
 [1.03226007e-01 4.47893526e-01]
 [9.08595503e-01 2.93614148e-01]
 [2.87775339e-01 1.30028572e-01]
 [1.93669579e-02 6.78835533e-01]
 [2.11628116e-01 2.65546659e-01]
 [4.91573159e-01 5.33625451e-02]
 [5.74117605e-01 1.46728575e-01]
 [5.89305537e-01 6.99758360e-01]
 [1

Epoch 521 : Total Loss 176.667
Epoch 531 : Total Loss 171.771
Epoch 541 : Total Loss 167.017
Epoch 551 : Total Loss 162.402
Epoch 561 : Total Loss 157.923
Epoch 571 : Total Loss 153.578
Epoch 581 : Total Loss 149.364
Epoch 591 : Total Loss 145.279
Epoch 601 : Total Loss 141.320
Epoch 611 : Total Loss 137.483
Epoch 621 : Total Loss 133.766
Epoch 631 : Total Loss 130.168
Epoch 641 : Total Loss 126.683
Epoch 651 : Total Loss 123.311
Epoch 661 : Total Loss 120.048
Epoch 671 : Total Loss 116.891
Epoch 681 : Total Loss 113.838
Epoch 691 : Total Loss 110.885
Epoch 701 : Total Loss 108.031
Epoch 711 : Total Loss 105.271
Epoch 721 : Total Loss 102.604
Epoch 731 : Total Loss 100.026
Epoch 741 : Total Loss 97.534
Epoch 751 : Total Loss 95.127
Epoch 761 : Total Loss 92.800
Epoch 771 : Total Loss 90.552
Epoch 781 : Total Loss 88.380
Epoch 791 : Total Loss 86.280
Epoch 801 : Total Loss 84.251
Epoch 811 : Total Loss 82.289
Epoch 821 : Total Loss 80.392
Epoch 831 : Total Loss 78.558
Epoch 841 : Total 

Epoch 3221 : Total Loss 8.118
Epoch 3231 : Total Loss 8.097
Epoch 3241 : Total Loss 8.075
Epoch 3251 : Total Loss 8.054
Epoch 3261 : Total Loss 8.033
Epoch 3271 : Total Loss 8.012
Epoch 3281 : Total Loss 7.991
Epoch 3291 : Total Loss 7.971
Epoch 3301 : Total Loss 7.951
Epoch 3311 : Total Loss 7.930
Epoch 3321 : Total Loss 7.910
Epoch 3331 : Total Loss 7.891
Epoch 3341 : Total Loss 7.871
Epoch 3351 : Total Loss 7.851
Epoch 3361 : Total Loss 7.832
Epoch 3371 : Total Loss 7.813
Epoch 3381 : Total Loss 7.794
Epoch 3391 : Total Loss 7.775
Epoch 3401 : Total Loss 7.757
Epoch 3411 : Total Loss 7.738
Epoch 3421 : Total Loss 7.720
Epoch 3431 : Total Loss 7.702
Epoch 3441 : Total Loss 7.684
Epoch 3451 : Total Loss 7.666
Epoch 3461 : Total Loss 7.648
Epoch 3471 : Total Loss 7.631
Epoch 3481 : Total Loss 7.614
Epoch 3491 : Total Loss 7.596
Epoch 3501 : Total Loss 7.579
Epoch 3511 : Total Loss 7.562
Epoch 3521 : Total Loss 7.546
Epoch 3531 : Total Loss 7.529
Epoch 3541 : Total Loss 7.513
Epoch 3551

In [33]:
# get our evaluation metrics

print(f'The runtime of the regular matrix factorization on a 100 x 100, 99% sparse matrix is {end4 - start4} seconds.')

mse, rmse = evaluation_metrics(arr_100_100, A_hat)

print(f'The mean squared error of all predictions is {mse:.3f}.')
print(f'The root mean squared error of all predictions is {rmse:.3f}.')

The runtime of the regular matrix factorization on a 100 x 100, 99% sparse matrix is 52.94888380006887 seconds.
The mean squared error of all predictions is 0.008.
The root mean squared error of all predictions is 0.089.


In [34]:
# get our predictions

mat_predict(arr_100_100, A_hat)

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 1.0,
 0.0,
 0.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 0.0,
 1.0,
 2.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 2.0,
 0.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 1.0,
 2.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 2.0,
 2.0,
 1.0,
 1.0,
 1.0,
 0.0,
 2.0,
 2.0,
 2.0,
 2.0,
 1.0,
 2.0,
 2.0,
 5.0,
 3.0,
 1.0,
 1.0,
 1.0,
 3.0,
 2.0,
 2.0,
 3.0,
 1.0,
 2.0,
 1.0,
 2.0,
 3.0,
 1.0,
 3.0,
 2.0,
 3.0,
 2.0,
 3.0,
 2.0,
 2.0,
 2.0,
 3.0,
 2.0,
 3.0,
 3.0,
 1.0,
 4.0,
 1.0,
 2.0,
 4.0,
 4.0,
 1.0,
 2.0,
 2.0,
 1.0,
 2.0,
 2.0,
 2.0,
 1.0,
 1.0,
 3.0,
 1.0,
 2.0,
 2.0,
 1.0,
 1.0,
 1.0,
 3.0,
 3.0,
 3.0,
 3.0,
 3.0,
 3.0,
 2.0,
 1.0,
 1.0,
 5.0,
 3.0,
 1.0

This is almost a 900% improvement in runtime! To see why, we can do a brief complexity analysis. In this case, looking at $\mathcal{O}$ is helpful because we are looking to factorize huge, sparse matrices.

### Complexity Analysis

The time complexity of the matrix factorization methods are not entirely clear as it depends on the cases that we are looking at.

Let us lay down some definitions:

**Sparsity (s)**:

$s$ is a parameter denoting the proportion of 0's in a matrix compared to the total number of entries. For an $m\times n$ interaction matrix, $A$, we see that the **sparsity of $A$** is:

$$s = \frac{\mathrm{Number \; of \; 0's}}{nm}$$

**Density (d)**:

Likewise, $d$ is a parameter denoting the proportion of nonzero's compared to the total number of entries. For $A$ indicated above, the **density of $A$** is:

$$d = \frac{\mathrm{Number \;of\;nonzero's}}{nm}$$

Sparsity and density are related by the following expression:

$$s = 1 - d$$

(rearrange to see density in terms of sparsity)

Now onto our analysis:

1) **Case 1: Dense Matrices**

There is no clear definition, but for our purposes, we consider a **dense matrix**, $A$ (assume $m\times n$) as one in which there is a **density** of at least $\frac{nm}{2}$, where the nonzero entries are distributed with little bias across all rows and columns.

Let $k$ denote the **number of latent features**, $n$ denote the **number of columns**, and $m$ the **number of rows**. Assume that our number of latent features is *fixed*, and is far smaller than the dimensions of our interaction table.  Assume that our training loop persists for $e$ **epochs** and that $e$ is *fixed* (in practice, the number of epochs can get large, but, in principle, it is fixed before we run our training loop).

Let $N := \max(m,n)$. For a general $m\times n$ ground truth, interaction matrix, $A$, the *regular matrix factorization* implementation has a big $\mathcal{O}$ time complexity of $\mathcal{O}(ekmn)$. In our case, since the latent features and epochs are fixed in size (the dimensions of the matrix are features of our data and model), we can consider $k << N$, $e << N$, and $m$ and $n$ to be arbitrarily large, and we see that the worst-case runtime is actually just $\mathcal{O}(N^{2})$.

Now look at the *sparse matrix factorization* implementation, note that this is no different than the *regular matrix factorization* as when we isolate the indices corresponding to nonzero entries, we, essentially, must traverse at least $0.5nm$ entries in $A$. Therefore, the time complexity of the sparse MF is just $\mathcal{O}(N^{2})$.

2) **Case 2: Sparse Matrices**

We consider a **sparse matrix** as one in which the **sparsity** is **at least** $1 - 1/\min(n,m)$ (i.e. the number of nonzero entries is approximately the number of rows or the number of columns). With notation above, we see that the worst-case runtime for the *sparse matrix factorization* is $\mathcal{O}(N)$, and the worst-case runtime for the *regular matrix factorization* is $\mathcal{O}(N^{2})$, and to see this, just recognize that $e$ and $k$ are fixed as above. 

In our *sparse matrix factorization*, since **we only traverse nonzero entries of the interaction table**, we are only running our training loop on at most $\max(n,m)$ entries. Therefore, since $k$ and $e$ are fixed, we get a time complexity of $\sim \mathcal{O}(N)$.

In our *regular matrix factorization*, since **we must traverse ALL entries of the interaction table**, we are running our training loop for $nm$ entries as we **must traverse the zero entries in order to check that they are zero**. Therefore, we have a time complexity of $\sim \mathcal{O}(N^{2})$.

### Why is Sparsity Important?

Sparsity is an important assumption about our input data that is actually true for the business problem. Oftentimes, when we are trying to analyze interactions, we will have many entries missing. Most people either do not leave ratings or the ratings are often not present due to some preprocessing method (like one-hot encoding). A sparse matrix is a very realistic and conservative assumption to make when we are trying to generate recommendations.

Not only does sparsity optimize our method above, but it is an assumption that often holds in the business problem where matrix factorization is relevant.